# Processing the raw data.

This jupyter notebook uses Google's Tesseract open source project to convert a pdf containing many charts into a collection of CSV files.

## Methodology

Attempts to use `tesseract` without any pre-Processing led to a lot of issues. To confront this I process the charts as follows:

  + Convert the pdf to a high-dpi png using Pillow.
  + Analyze the pixels of the png. Search for vertical straight lines to identify columns.
  + Chop the png into strips, where each strip is a column of data.
  + Run the column through tesseract. Tesseract converts it to a column of text.
  + Repeat for each strip, building a representation of the data in a format that can be exported to CSV. Note that these are still strings. `tesseract` is fairly accurate but often gets the commas and stray marks in the data miss-interperited.
  + Lastly, I look at the CSV, look for mistakes, and correct them.

In [1]:
# Import block.
import pytesseract
import csv
from PIL import Image
import numpy as np
# %matplotlib notebook
from matplotlib import pyplot as plt


In [2]:
# Convert a specific page of the pdf to png.
import pdf2image
def getPngPage(index: int) -> Image:
    filePath = './colonial-1970-migration.pdf'
    tmp = pdf2image.convert_from_path(
        filePath,
        output_file='./colonial_%d' % index,
        dpi=600,
        first_page=index,
        last_page=index,
        grayscale=True
    )[0]
    tmpData = np.asarray(tmp)
    return Image.fromarray((tmpData > 128).astype(np.uint8)*255)


In [3]:
# Algorithm for finding what's a good threshold of pixel values to identify column divisions in the graph.
def findThreshold() -> int:
    chart1 = getPngPage(19)
    rawData = np.asarray(chart1)
    height, width = rawData.shape
    summed = [rawData[:,i].sum() for i in range(width)]
    return 800000/(height*255)
threshold = findThreshold()
# Found a reasonable threshold of less than 800,000

def extractColumnsInds(data: np.ndarray) -> [[int]]:
    # First get all columns below threshold
    height, width = data.shape
    summedValues = np.array([data[:,i].sum()/(height*255) for i in range(width)])
    lineIndices = np.where(summedValues < threshold)[0]
    # Now I need to remove duplicates - perhaps there were two pixel columns that passed the threshold that are really one table column.
    ret = []
    colStart, colEnd = (0, 0)
    for index in range(len(lineIndices)-1):
        nextInd = index + 1
        if lineIndices[nextInd] > lineIndices[index] + 200:
            ret.append([lineIndices[index], lineIndices[nextInd]])
    return ret

def getRowInds(data: np.ndarray) -> [int]:
    height, width = data.shape
    rowThreshold = 0.97
    rowSums = np.array([data[i,:].sum()/(width*255) for i in range(height)])
    aboveThreshold = np.where(rowSums > rowThreshold)[0]
    ret = []
    summation = 0
    counter = 1
    for index in range(len(aboveThreshold)-1):
        if aboveThreshold[index + 1] == aboveThreshold[index] + 1:
            counter += 1
            summation += aboveThreshold[index]
        else:
            ret.append(int(summation/counter))
            counter = 1
            summation = 0
    return ret

def removeHeader(data: Image) -> Image:
    npData = np.asarray(data)
    height, width = npData.shape
    topThird = npData[:height // 3,:]
    sums = np.array([topThird[i,:].sum() for i in range(topThird.shape[0])])
    lastLine = np.where(sums < 20000)[0].max()
    return Image.fromarray(npData[lastLine:,:])


In [4]:
tesseractConfig = "--psm 13 --oem 1 -c tessedit_char_whitelist=0123456789.,`\\'\\\""

def getTextData(inputImage: Image) -> np.ndarray:
    totalData = np.asarray(inputImage)
    colInds = extractColumnsInds(totalData)
    colImgs = [Image.fromarray(totalData[:,s+1:e]) for s, e in colInds]
    dataDump = np.ndarray((1000,1000), dtype=object)
    for colI, col in enumerate(colImgs):
        noHeader = removeHeader(col)
        colData = np.asarray(noHeader)
        rowInds = getRowInds(colData)
        for ind in range(len(rowInds) - 1):
            imgSlice = colData[rowInds[ind]:rowInds[ind+1],:]
            try:
                img = Image.fromarray(imgSlice)
                textCell = pytesseract.image_to_string(img, config=tesseractConfig)
                dataDump[ind, colI] = textCell
            except:
                print("EXCEPTION AT (%d,%d)" % (colI, ind))
    return dataDump


def getCsv(pageNumber: int) -> [[str]]:
    totalImage = getPngPage(pageNumber)
    listOfLists = getTextData(totalImage)
    return listOfLists
    dataDump = np.ndarray((1000, 1000), dtype=object) # Constants from the page in question.
    with open('chart_%d.csv' % pageNumber, 'w+') as file:
        writer = csv.writer(file)
        for i, col in enumerate(listOfLists):
            for j, row in enumerate(col):
                dataDump[j,i] = listOfLists[i][j]
        for rowI in range(80):
            writer.writerow(list(dataDump[rowI]))
    return dataDump


In [6]:
getCsv(19)

EXCEPTION AT (0,20)
EXCEPTION AT (0,22)
EXCEPTION AT (0,25)
EXCEPTION AT (0,30)
EXCEPTION AT (0,35)
EXCEPTION AT (0,40)
EXCEPTION AT (0,45)
EXCEPTION AT (0,50)
EXCEPTION AT (0,55)
EXCEPTION AT (0,57)
EXCEPTION AT (0,60)
EXCEPTION AT (0,65)
EXCEPTION AT (0,68)
EXCEPTION AT (0,70)
EXCEPTION AT (0,73)
EXCEPTION AT (0,75)
EXCEPTION AT (0,76)
EXCEPTION AT (1,15)
EXCEPTION AT (1,20)
EXCEPTION AT (1,25)
EXCEPTION AT (1,30)
EXCEPTION AT (1,35)
EXCEPTION AT (1,36)
EXCEPTION AT (1,40)
EXCEPTION AT (1,45)
EXCEPTION AT (1,48)
EXCEPTION AT (1,50)
EXCEPTION AT (1,52)
EXCEPTION AT (1,55)
EXCEPTION AT (1,56)
EXCEPTION AT (1,60)
EXCEPTION AT (1,62)
EXCEPTION AT (1,65)
EXCEPTION AT (1,68)
EXCEPTION AT (1,70)
EXCEPTION AT (1,71)
EXCEPTION AT (1,73)
EXCEPTION AT (1,75)
EXCEPTION AT (2,14)
EXCEPTION AT (2,17)
EXCEPTION AT (2,20)
EXCEPTION AT (2,25)
EXCEPTION AT (2,28)
EXCEPTION AT (2,31)
EXCEPTION AT (2,36)
EXCEPTION AT (2,42)
EXCEPTION AT (2,45)
EXCEPTION AT (2,47)
EXCEPTION AT (2,52)
EXCEPTION AT (2,57)
